# Course 2: Sensor Fusion
## Part 4: Unscented Kalman Filter (UKF)
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Work out the [Unscented Kalman filter](https://en.wikipedia.org/wiki/Kalman_filter#Unscented_Kalman_filter) (UKF) predict and innovation / update equations;
* Understand and apply the Constant Turn Rate and Velocity Magnitude (CTRV) motion model to our problem formulation;
* Implement a radar sensor measurement and noise model;
* Apply the UKF algorithm to the CTRV model and perform state / covariance estimation prediction using nonlinear functions. 

## 1. Introduction

### 1.1. Unscented Kalman Filter (UKF)

#### Background

In this lesson, we will be building on our knowledge of the [extended Kalman filter](https://en.wikipedia.org/wiki/Extended_Kalman_filter) (EKF) [1] in order to implement an [Unscented Kalman filter](https://en.wikipedia.org/wiki/Kalman_filter#Unscented_Kalman_filter)
 (UKF) [2][3] for object tracking with non-linear process and measurement models.
 
Rather than linearising a non-linear process model, the Unscented Kalman filter [3] use a deterministic sampling technique known as an [unscented transformation](https://en.wikipedia.org/wiki/Unscented_transform) to pick a minimum set of sample points (known as sigma points) around the mean of the covariance. As a bit of history, this technique was first proposed in the Julier 1997 [2] paper as an efficient means of representing the probability distribution of a system in a Kalman filter. The paper by Wan et al., 2000 [3] builds on this by introducing a full definition of the Unscented Kalman filter (UKF) which uses a nonlinear transformation of sigma points to more accurately approximate the system's probability distribution. With these sigma point estimates we evaluate the original non-linear process model $\mathcal{f}$ to estimate a new mean and covariance. The UKF approach is often more accurate than the extended Kalman filter (EKF) and avoids the computation of the Jacobians, which for some nonlinear high-dimensional / non-differentiable models is not possible.

The origin of the UKF's name is somewhat up for debate, but one theory from Sebastian Thrun suggests that the filter was named out of distaste for the extended Kalman filter's linearisation technique. The EKF's linearisation about a single working point, and the need for Jacobian derivations, was said to be considered a "stinky" approach by the UKF's authors. Therefore, the UKF provided a (euphemistically) "unscented", i.e., more appealing, technique which better estimated a non-linear distribution. An alternative take on the UKF's name by Hao Li in their 2014 pre-print [4] suggests that the name is itself rooted in the "scented" process (verb: to _impart_ a pleasant scent onto) by which the extended Kalman filter emitted state estimations. Rather than gradually "imparting" (losing) desirable information about the true state of the object being estimated, the UKF provides an "unscented" (and presumably lossless) emission of state estimates without the loss of the true statistics of the object's actual state.

#### From Extended- to Unscented Kalman Filter

The UKF uses the same prediction and update equations found in the extended Kalman filter [1]. The difference between the two, however, is how the UKF deals with non-linear process and measurement models. Rather than linearising the non-linear functions using a first-order Taylor expansion and deriving a Jacobian matrix, we instead sample a set of "sigma points" from a Gaussian distribution centred about the mean of the covariance given for the non-linear process model.

#### UKF Process Chain

Given a state vector mean $x_{k\vert k}$ and state covariance $\mathrm{P}_{k\vert k}$ at time-step $k$. Our goal with the UKF is to predict the mean state $x_{k+1 \vert k}$ and covariance $\mathrm{P}_{k+1\vert k}$ to the next time-step $k+1$. Note that here we use the pipe "$\vert$" notation to represent on the left-hand side the current estimated time-step, and on the right-hand side the latest considered measurement. For $x_{k\vert k}$ and $\mathrm{P}_{k\vert k}$, we say that the estimation is for time-step $k$ and the measurement at time-step $k$ has already been taken into account — this is commonly referred to as the _posterior estimation_. On the other hand, for $x_{k+1\vert k}$ and $\mathrm{P}_{k+1\vert k}$, we have the estimation for time-step $k+1$ which takes into account the measurement from the previous time-step $k$ — this is commonly referred to as the _a priori estimation_.

<img src="figures/Exercises/2023-01-13-Figure-1-Unscented-Kalman-Filter-Estimation.png" alt="Figure 1. The Unscented Kalman Filter — Posterior and A priori distributions formed by the state mean and covariance before- and after the estimation at time-step k+1.">

$$
\begin{align}
\textrm{Figure 1. The Unscented Kalman Filter — Posterior and A priori distributions formed by the state mean and covariance before- and after estimation at time-step k+1.}
\end{align}
$$

The two ellipses in Figure 1 above visualise the distribution of the uncertainty; all points along the perimeter of the ellipse have the same probability density. If the uncertainty is normally-distributed these points form the shape of the ellipse, as shown above (often called the "error ellipse"). The _A priori_ estimation is what is obtained after the prediction step of the UKF has been performed. These two ellipses can be seen as the visualisation of the covariance matrix $\mathrm{P}$. Assuming a linear process model, we have a prediction step given by the equations:

$$
\begin{align}
x_{k+1} &= \mathrm{F}x_{k} + \nu_{k}, \\
\mathcal{Q} &= E\left[\nu_{k} \cdot \nu_{k}^{\top}\right].
\end{align}
$$

Here, $\mathcal{Q}$ is the covariance matrix of the process noise. In the linear case, the Kalman filter is used to solve the prediction problem. A solution can be found with the following set of equations:


$$
\begin{align}
x_{k+1 \vert k} &= \mathrm{F}x_{k \vert k}, \\
\mathrm{P}_{k+1 \vert k} &= \mathrm{F}\mathrm{P}_{k\vert k}\mathrm{F}^{\top} + \mathcal{Q}.
\end{align}
$$

When the process model is non-linear, as in the case of our problem, we have the prediction step given as:

$$
\begin{align}
x_{k+1} &= \mathcal{f}\left(x_{k} + \nu_{k}\right), \\
\mathcal{Q} &= E\left[\nu_{k} \cdot \nu_{k}^{\top}\right].
\end{align}
$$

Since the prediction step of a non-linear process model results in a non-linear distribution, it is no longer possible to directly represent the predicted state distribution, i.e., the a priori estimation, as a normal distribution forming an ellipse. We no longer have an analytical path, and therefore we need to find an approximate normal distribution that represents the original non-linear distribution as closely as possible. In order to do so, we find a normal distribution characterised by the same mean value and the covariance matrix as the _real_ predicted distribution. The question is, how do we derive this mean vector and this covariance matrix? This is what the unscented transformation does for us.

###### Unscented Transformation

To deal with non-linear functions, the UKF uses an [unscented transformation](https://en.wikipedia.org/wiki/Unscented_transform). The unscented transformation provides the mean state vector and covariance matrix needed to estimate the a priori normal distribution. By using sigma points, we are able to approximate the non-linear distribution using only a set of point estimates. These point estimates, called sigma points, are transformed into the predicted state-space using the non-linear process model function $\mathcal{f}$. The sigma points are chosen relative to the mean state and are defined with respect to the standard deviation signal of every state dimension. This is why these points are called _sigma_ points; they serve as representatives of the whole distribution.

Once the set of sigma points has been transformed into the predicted state-space by the non-linear function $\mathcal{f}$, all that is left to do is calculate the mean and covariance of the sampled sigma points. The mean and covariance of the sigma points serves as only an approximation of the _real_ predicted distribution — but in most cases we get a useful estimation to work with.

Another great thing about the unscented transformation and sigma point estimation is that, in the linear case, the _exact_ solution to the real predicted distribution can be derived. Thus, for a linear process model, the sigma points provide the exactly the same solution as the standard Kalman filter. Pretty neat, huh? While all this sounds great, using sigma points in the linear case isn't an entirely sound decision, since their added complexity weighs things down in terms of calculation time.

As a recap, sigma points provide a way to estimate a non-linear distribution, providing the best possible mean and covariance estimates for a normal distribution assumption. Now, with the general idea of sigma points out of the way, let's dive into the meat of the Unscented Kalman filter — the _prediction_ and _update_ steps.

###### Prediction and Innovation of Sigma Points

Starting with a state mean vector $x_{k \vert k}$ and covariance $\mathrm{P}_{k \vert k}$, the prediction and update (also called _innovation_) steps of the UKF give us the updated state vector $x_{k+1 \vert k}$ and covariance matrix $\mathrm{P}_{k+1 \vert k}$ predicted into the next time-step $k+1$. This is essential to applications such as object tracking where we must reliably estimate the change in vehicle position and orientation over a given time period $\Delta t$.

Since we begin the UKF process chain with the _prediction_ step, we split this into three parts: first, we need to know a good way to choose the sigma points. Secondly, we need to know how to predict the sigma points into the actual state-space of the process model. Lastly, we need to know how to calculate the prediction mean and covariance from the sigma point estimations.

We start with prediction off with the posterior state $x_{k\vert k}$ and the posterior covariance matrix $\mathrm{P}_{k \vert k}$ of the vehicle from the previous iteration. They represent the distribution of our vehicle's current state. This distribution is a linear, normal distribution from which we can generate our sigma points. The number of sigma points we generate at this step depends on the dimension of the state vector $n_{x}$. Since we describe in Sect 1.2. the CTRV model with state vector of dimension $5$, we say here that $n_{x} = 5$. We choose a number of sigma points

$$
\begin{align}
n_{\sigma} &= 2*n_{x} + 1 \\
\end{align}
$$

which, for the CTRV model gives us a total of $n_{\sigma} = 2*\left(5\right) + 1 = 11$ points to generate. This corresponds to _one_ sigma point to represent the mean of the state, and another two sigma points for each dimension of the state. Each of these two sigma points are spread in opposite directions, and their "spread" is determined by a design parameter $\lambda$ which governs their distance from the centre of the distribution (the mean point). Assuming a state vector of only the position $x_{k} = \left[p_{x} p_{y}\right]$, we can illustrate the rule for the sigma point matrix as:

$$
\begin{align}
\mathcal{X}_{k\vert k} &= \begin{bmatrix} x_{k\vert k} & x_{k\vert k} + \sqrt{\left(\lambda + n_{x}\right)\mathrm{P}_{k\vert k}} & x_{k \vert k} - \sqrt{\left(\lambda + n_{x}\right)\mathrm{P}_{k\vert k}}\end{bmatrix},
\end{align}
$$

such that the columns of the sigma point matrix $\mathcal{X}_{k\vert k}$ contain the set of $n_{\sigma} = 5$ sigma points corresponding to the state dimension of the position coordinates of $n_{x} = 2$. The "spreading factor" in the square-root $\left(\lambda + n_{x}\right)$ governs the spread of the sigma points, where $\lambda = 3 - n_{x}$ is commonly chosen to compute this value. While it isn't directly relevant here, we will be using the [Cholesky decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition#Kalman_filters) to compute the square-root of the covariance matrix $\mathrm{P}_{k \vert k}$ as given by the decomposition $\mathrm{A} = \mathrm{L}\mathrm{L}^{\top}$ where the columns of $\mathrm{L}$ are added and subtracted to the mean $x$ to form a set of $2*n_{x}$ vectors. Here, $\mathrm{L}$ is the [lower triangular matrix](https://en.wikipedia.org/wiki/Lower_triangular_matrix) and, when $\mathrm{A}$ is assumed to be a real (hence, symmetric positive-definite) matrix, $\mathrm{L}$ is has real-valued positive entries along the diagonal. Also important to note for our later implementation of this sigma point generator function is that the first column of the matrix $\mathcal{X}_{k\vert k}$ is the mean of the distribution $x_{k\vert k}$, the second through $n_{x} + 1$ column will be given by the second term above (with the addition sign between the mean and the square-root term), and the last $n_{x} + 2$ through $2 * n_{x} + 1$ columns will be given by the third term above (with the subtraction sign between the mean and the square-root term).

##### UKF Augmentation

Now that we know how to represent the uncertainty of the posterior state estimation using sigma points, accomplishing the second step of the UKF _prediction_ is nearly complete. Just plug in each sigma point into the non-linear process function $\mathcal{f}$ to obtain the next-state estimation:

$$
\begin{align}
x_{k+1} &= \mathcal{f}\left(x_{k}, \nu_{k}\right).
\end{align}
$$

However, we have yet to consider the effect of the non-linear process noise vector $\nu_{k}$. Fortunately, with the UKF we are able to handle non-linear process noise in a step known as _UKF augmentation_. To start, we recall the expression of the independent noise processes as stochastic properties (this will be covered in the next section for the CTRV model). From here, we write the process noise covariance matrix as the expression:

$$
\begin{align}
\mathcal{Q} &= E\left\{\nu_{k}\cdot \nu_{k}^{\top}\right\} = \begin{bmatrix}\sigma_{a}^{2} & 0 \\ 0 & \sigma_{\ddot{\psi}^{2}}\end{bmatrix},
\end{align}
$$

which is a matrix that contains the variances of the noise processes for the longitudinal acceleration and change in yaw angle. Here, we consider the white-noise processes independent from each other (hence, the $0$ values along the diagonal).

In the augmentation step, we form the _augmented_ state $x_{a}$ by adding the noise vector $\nu_{k}$ to the state vector $x_{k}$. Repeating the sigma point generation for this augmented state vector $x_{a, k}$ and covariance matrix $\mathrm{P}_{a, k \vert k}$, we obtain a new set of sigma points representing the uncertainty caused by the sigma points. Note that the number of augmented sigma points computed here differs than in the first calculation, since we have an additional two terms added to the augmented state vector, resulting in the dimensions $n_{a} = n_{x} + 2$. From our earlier example, we obtain $n_{\sigma_{a}} = 2*7 + 1 = 15$ augmented sigma points. This results in an augmented covariance matrix $\mathrm{P}_{a, k\vert k}$ of dimensions $7 \times 7$ such that

$$
\begin{align}
\mathrm{P}_{a, k\vert k} &= \begin{bmatrix} \mathrm{P}_{k, \vert k} & 0 \\ 0 & \mathcal{Q} \end{bmatrix},
\end{align}
$$

where the upper-left of the matrix is given as the original covariance matrix, and the lower-right of the matrix is the independent process noise covariance matrix defined above. The remaining values in the matrix are therefore set to zero.

Lastly, we compute these sigma points in the exact same way as before. Just evaluate the expression for the previous sigma point matrix to get the augmented sigma point matrix $\mathcal{X}_{a, k\vert k}$. No need to modify the scaling factor on this one. Once the augmented sigma point matrix has been formed, simply plug these points into the process model $\mathrm{f}$ using the process noise vector $\nu_{k}$ to obtain the predicted sigma point matrix $\mathcal{X}_{k+1\vert k}$. You'll see in the following Sect. 1.2 the derivation of the state-space form of the process model $\mathcal{f}$ and the expression for the next-state prediction, which we will evaluate in order to obtain this matrix.

##### Prediction and Innovation of Mean and Covariance

Before jumping into the innovation step of the UKF, we finish off the prediction step by calculating the mean and covariance of the predicted state from the augmented sigma points we just derived. The standard rule for calculating the mean and covariance of a group of state samples is given by the following equations:

$$
\begin{align}
x_{k+1\vert k} &= \sum_{i=0}^{n_{a}} w_{i}\mathcal{X}_{k+1\vert k, i}, \\
\mathrm{P}_{k+1 \vert k} &= \sum_{i=0}^{n_{a}} w_{i} \left(\mathcal{X}_{k+1\vert k, i} - x_{k+1\vert k}\right)\left(\mathcal{X}_{k+1\vert k, i} - x_{k+1\vert k}\right)^{\top}.
\end{align}
$$

Here, we compute the sum over the columns of the predicted sigma point matrix (hence, the sum from $i=0$ to $i=n_{a}$ where $n_{a}$ corresponds to the number of augmented sigma points). In these expressions is the use of a weight vector $w$ with values given by:

$$
\begin{align}
w_{i} &= 
\begin{cases}
    \frac{\lambda}{\lambda + n_{\mathrm{aug}}}, & \text{if} \quad i = 0 \\
    \frac{1}{2}\left(\lambda + n_{\mathrm{aug}}\right), & \text{otherwise}. \\
\end{cases}
\end{align}
$$

where the weight expression differs for the first element of the vector and all other elements $i = 1,\ldots, n_{\mathrm{aug}}$. Note that the dimension $n_{\mathrm{aug}}$ is the dimension of the augmented state vector, which previously we assumed to be $n_{\mathrm{aug}} = n_{x} + 2$. These weight values also depend on the spreading parameter $\lambda$ which is used in the sigma point generation. This is to recover the covariance by inverting the spreading of the sigma points. There's more than one way to define these weights in literature, but for our implementation we use the expressions above.

##### Prediction of the Measurement

We have successfully arrived at the first step of the UKF _innovation_ / update process. Here, our goal is to transform the predicted state into the measurement space. The function that defines this transformation is the _measurement model_. Of course, here is now where we must consider the type of measurement we are transforming the predictions with respect to. For our later implementation we will be using a radar sensor model whose measurement vector $z_{k+1\vert k}$ and expression of the measurement function $\mathcal{h}$ are given as:

$$
\begin{align}
z_{k+1\vert k} &= \begin{bmatrix} \rho & \phi & \dot{\rho} \end{bmatrix}^{\top}, \\
\
z_{k+1} &= \mathcal{h}\left(x_{k+1}\right) + \omega_{k+1}.
\end{align}
$$

Here, the $\rho$ term is the measured radial distance ($\mathrm{m}$), the $\phi$ term is the heading angle ($\mathrm{rad}$), and $\dot{\rho}$ as the rate-of-change of radial distance (i.e., velocity, $\mathrm{m}/\mathrm{s}^{2}$).

This transformation process is very similar to the problem we had in the prediction step; we need to transform a distribution through a non-linear function $\mathcal{h}$. Therefore, we apply the exact same unscented transformation approach as we did before during the state prediction. However, this time we take two shortcuts here: (1) we repeat the sigma point generation using the predicted mean and covariance matrix, and (2) hold out computing the effect of measurement noise on the radar model.

Along the path of shortcut (1) we re-use the sigma points we have already generated from the prediction step and skip generating new sigma points all-together for the radar measurement. This allows us to also skip the augmentation step entirely with shortcut (2), since we assume here a purely _linear_ additive effect of measurement noise on the radar measurement model. This gives an easier way to consider measurement noise which we will jump back to later in the calculation of the measurement covariance matrix using matrix $\mathrm{R}$. So for now, set the $\omega_{k+1}$ term to be $0$.

With these two simplifications, we only have to transform the individual sigma points we previously derived into the radar measurement space. Then, we use them to calculate the mean $z_{k+1\vert k}$ and covariance $\mathrm{S}$ of the predicted radar measurement. The re-used sigma points, after using them to evaluate the measurement model $\mathcal{h}$, are stored in the radar sigma point matrix $\mathcal{Z}_{k+1\vert k}$.

Applying the equations to our original problem formulation, we obtain a predicted sigma point matrix $\mathcal{X}_{k+1 \vert k}$ of dimensions $5 \times 15$ and a transformed sigma point matrix into measurement space $\mathcal{Z}_{k+1\vert k}$ of dimensions $3 \times 15$. The last step we have to do involving these sigma point matrices is to compute the predicted measurement mean $z_{k+1 \vert k}$ and the predicted measurement covariance $\mathrm{S}_{k+1\vert k}$, as follows:

$$
\begin{align}
z_{k+1\vert k} &= \sum_{i=1}^{n_{\sigma}} w_{i}\mathcal{Z}_{k+1\vert k, i}, \\
\mathrm{S}_{k+1\vert k} &= \sum_{i=0}^{n_{\sigma}} w_{i}\left(\mathcal{Z}_{k+1 \vert k, i} - z_{k+1\vert k}\right)\left(\mathcal{Z}_{k+1\vert k, i} - z_{k+1\vert k}\right)^{\top} + \mathrm{R}. \\
\end{align}
$$

In the above prediction equations we have the measurement noise covariance $\mathrm{R}$ which follows $\mathrm{R} = E\{\omega_{k} \cdot \omega_{k}^{\top}\}$. This applies to our case since the measurement noise of the radar sensor is linear and has a purely additive effect on the measurement model. Therefore, we obtain the covariance as the following:

$$
\begin{align}
\mathrm{R} &= E\{\omega_{k} \cdot \omega_{k}^{\top}\} = \begin{bmatrix}\sigma_{\rho}^{2} & 0 & 0 \\ 0 & \sigma_{\phi}^{2} & 0 \\ 0 & 0 & \sigma_{\dot{\rho}}^{2}\end{bmatrix}.
\end{align}
$$

##### UKF Update Step

We are finally at the last step of the UKF. This step is needed to update the state and covariance matrix using the received measurement. To recap, we have derived the predicted state mean $x_{k+1\vert k}$ and the predicted covariance $\mathrm{P}_{k+1\vert k}$, as well as the predicted measurement mean $z_{k+1\vert k}$ and measurement covariance $\mathrm{S}_{k+1\vert k}$. At time-step $k+1$ we receive a radar measurement $z_{k+1}$ which we will now use to update these four terms. Assuming we know the incoming measurement sensor type, we evaluate the respective measurement model and look into the measurement values. With this, we proceed to evaluate the standard Kalman filter update step equations but with just a slight caveat — the expression for the Kalman gain $\mathrm{K}$ and consequently the state update $x_{k+1\vert k+1}$ and covariance matrix update $\mathrm{P}_{k+1\vert k+1}$ rely on the computation of a cross-correlation matrix $\mathrm{T}_{k+1\vert k}$. This cross-correlation matrix is given as the difference between the sigma points defined in state-space $\mathcal{X}_{k+1\vert k}$ and the sigma points in measurement space $\mathcal{Z}_{k+1\vert k}$ and is as follows:

$$
\begin{align}
T_{k+1\vert k} &= \sum_{i=0}^{n_{\sigma}} w_{i}\left(\mathcal{X}_{k+1\vert k, i} - x_{k+1\vert k}\right)\left(\mathcal{Z}_{k+1\vert k, i} - z_{k+1\vert k}\right)^{\top},
\end{align}
$$

which is used in the standard Kalman filter update equations:

$$
\begin{align}
\mathrm{K}_{k+1\vert k} &= \mathrm{T}_{k+1\vert k}\mathrm{S}^{-1}_{k+1\vert k}, \\
x_{k+1 \vert k+1} &= x_{k+1\vert k} + \mathrm{K}_{k+1\vert k} \left(z_{k+1} - z_{k+1\vert k}\right), \\
P_{k+1\vert k+1} &= \mathrm{P}_{k+1\vert k} - \mathrm{K}_{k+1 \vert k}\mathrm{S}_{k+1\vert k}\mathrm{K}^{\top}_{k+1\vert k},
\end{align}
$$

which are the expressions for the Kalman gain $\mathrm{K}_{k+1\vert k}$, the state update $x_{k+1\vert k+1}$, and the covariance matrix update $\mathrm{P}_{k+1\vert k+1}$, respectively.

##### Summary

Congratulations! If you made it this far, you know have every equation and every detail needed to dive right into the implementation of the UKF in C++. While this was a long, enduring endeavour, I promise that the equations above serve almost their exact definitions when translated into C++ code (thanks to our use of the [Eigen](https://en.wikipedia.org/wiki/Eigen_\(C%2B%2B_library\)) matrix manipulation library). 

Ready to take on this challenge with us in Sect. 2.1?

### 1.2. Constant Turn Rate and Velocity Magnitude (CTRV) Model

#### Background

As with most constant velocity models, the Constant Turn Rate and Velocity Magnitude (CTRV) model is a simplification of how objects actually navigate in real-world environments. The CTRV model assumes that an object (here, a vehicle), moves along a path with a constant turn-rate and constant velocity. Rather than a strict constant velocity, constant direction model, we allow the vehicle to travel a curved path with a constant heading / yaw angle. This additional assumption allows us to represent vehicle motion on roads with turns.  

We will see in Figure 1 in the later part of this section that here we approximate the vehicle's constant turn-rate $\psi$ with respect to a constant velocity $v$. In Figure 1 this is shown as a line tangential to the vehicle's actual (assumed) motion along a curved path. For now, just know that the CTRV model provides a decent starting point for representing vehicle behaviour in real traffic scenarios.

#### State equations

Using the CTRV model, a vehicle state at a given point in time can be represented with the following state vector: 

$$
\begin{align}
x = \begin{bmatrix}p_{x} & p_{y} & v & \psi & \dot{\psi}\end{bmatrix}^{\top},
\end{align}
$$

where the 2D vehicle position is given by $\left(p_{x}, p_{y}\right)$, the magnitude of velocity $v$, the orientation $\psi$ (i.e., the yaw angle of the velocity component), and the rate-of-change of the orientation $\dot{\psi}$.

##### Process model
In order to represent a state change of the vehicle from time-step $k$ to $k+1$, we use the transition function:
$$
\begin{align}
x_{k+1\vert k} &= \mathcal{f}\left(x_{k}, \nu_{k}\right).
\end{align}
$$

The goal of the process model function represented above is to predict the state of the vehicle $x_{k+1 \vert k}$ at the new time-step. To evaluate the process model $\mathcal{f}$, we use the current vehicle state $x_{k}$ and the process noise $\nu_{k}$ as inputs. In order to compute the state change, we must form an expression for the change-rate of the state $\dot{x}$, given as:
$$
\begin{align}
\dot{x} &= \begin{bmatrix}\dot{p}_{x} & \dot{p}_{y} & \dot{v} & \dot{\psi}, & \ddot{\psi} \end{bmatrix}^{\top}.
\end{align}
$$

From this relation we can directly derive the change-rate $\dot{x}$ in terms of a differential equation $\dot{x} = g\left(x\right)$.

We can re-write each of the terms of the state differential independently of any of the state elements in $x$. For example, we can write $\dot{p}_{x}$, i.e., the change in velocity along the $x$-axis, as:
$$
\begin{align}
\dot{p}_{x} = v_{x} = \cos(\psi)\cdot v,
\end{align}
$$

assuming a constant direction from the current vehicle state $x_{k}$ and the predicted vehicle state $x_{k+1}$. With this, we form a trigonometric relation an derive the expressions of the other state variables as follows:


$$
\begin{align}
\dot{x} &= 
\begin{bmatrix} \dot{p}_{x} \\ \dot{p}_{y} \\ \dot{v} \\ \dot{\psi} \\ \ddot{\psi} 
\end{bmatrix} = 
\begin{bmatrix}
\cos(\psi) \cdot v \\ \sin(\psi) \cdot v \\ 0 \\ \dot{\psi} \\ 0 
\end{bmatrix}.
\end{align}
$$

Since we assume a constant turn-rate and a constant velocity, our expression for the rate-of-change of both these state variables is therefore equal to $0$.

In order to form an expression for the state change over time, we must define our time-step $\Delta t$ as the change in time from $t_{k}$ to $t_{k+1}$. Our expression for the next-state $x_{k+1}$ is therefore:
$$
\begin{align}
x_{k+1} &= x_{k} + \int_{t_{k}}^{t_{k+1}} \begin{bmatrix} \dot{p}_{x}(t), \  \dot{p}_{y}(t), \ \dot{v}(t), \ \dot{\psi}(t), \ \ddot{\psi}(t) \end{bmatrix}^{\top}dt.
\end{align}
$$

Expanding the integral and replacing the derivative terms in the next-state vector, then solving for the integral terms, we obtain:

$$
\begin{align}
x_{k+1} &= x_{k} + \begin{bmatrix}v_{k}\int_{t_{k}}^{t_{k+1}} \mathrm{cos}\left(\psi_{k} + \dot{\psi} \cdot\left(t - t_{k}\right)\right) dt 
\\ v_{k}\int_{t_{k}}^{t_{k+1}} \mathrm{sin}\left(\psi_{k} + \dot{\psi} \cdot\left(t - t_{k}\right)\right) dt 
\\ 0 
\\ \dot{\psi}_{k}\cdot \Delta t 
\\ 0 \end{bmatrix}
= 
\begin{bmatrix}
\frac{v_{k}}{\dot{\psi}_{k}}\left(\mathrm{sin}\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) - \mathrm{sin}\left(\psi_{k}\right)\right)
\\ \frac{v_{k}}{\dot{\psi}_{k}}\left(-\mathrm{cos}\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) + \mathrm{cos}\left(\psi_{k}\right)\right)
\\ 0
\\ \dot{\psi}_{k} \cdot \Delta t
\\ 0
\end{bmatrix}.
\end{align}
$$

When the rate-of-change of the yaw angle $\dot{\psi}_{k} = 0 \ \mathrm{rad}/\mathrm{s}^{2}$, the first two terms of the final state vector are reduced to $v_{k}\cos\left(\psi_{k}\right)\cdot \Delta t$ and $v_{k}\tan\left(\psi_{k}\right)\cdot \Delta t$, respectively. 

<img src="figures/Exercises/2023-01-13-Figure-2-Constant-Turn-Rate-Constant-Velocity-Model.png" alt="Figure 2. The Constant Turn Rate Constant Velocity (CTRV) model visualised.">

$$
\begin{align}
\textrm{Figure 2. The Constant Turn Rate and Velocity Magnitude model.}
\end{align}
$$

In the above figure we depict the CTRV model assumption applied to the vehicle state at time-step $k$ to time-step $k+1$. The state transition is given by the process model $f$ which predicts the next-state of the vehicle at time $k+1$ using a constant velocity assumption — that is, assuming the vehicle proceeds to the next time-step with a constant heading angle $\psi$ and a constant velocity $v$. The CTRV model assumption allows us to simplify the calculation of the velocity to its magnitude rather than in either the $x$- and $y$-direction. The CTRV model gives us a basis for the estimation by exploiting the geometry of the assumed next-state trajectory. With a rate-of-change of the yaw angle $\ddot{\psi}$, our state transition equation with vector $\dot{x}$ becomes:

$$
\begin{align}
x_{k+1} = x_{k} + 
\begin{bmatrix}
v_{k}\cos\left(\psi_{k}\right)\Delta t \\
v_{k}\sin\left(\psi_{k}\right)\Delta t \\
0 \\
0 \\
0 \\
\end{bmatrix},
\end{align}
$$

which completes our definition of the deterministic part of the process model.

##### Uncertainty of the process model

The stochastic part of the process model is defined with respect to a process noise vector $\nu_{k} = \left[\nu_{a, k} \nu_{\ddot{\psi}, k}\right]^{\top}$, which describe the two independent scalar noise processes. The first noise process is the longitudinal acceleration noise $\nu_{a, k}$ which influences the longitudinal speed of the vehicle and it randomly changes its value at every time step $k$. The longitudinal acceleration is a normally-distributed white-noise with zero-mean and variance given by $\sigma_{a}^{2}$. The other noise process is the yaw acceleration noise $\nu_{\ddot{\psi}}$ which is also a normally-distributed white-noise with zero-mean and a variance given by $\sigma_{\ddot{\psi}}^{2}$, i.e., 
$$
\begin{align}
\nu_{a, k} &\sim \mathcal{N}\left(0, \sigma_{a}^{2}\right), \\
\nu_{\ddot{\psi}, k} &\sim \mathcal{N}\left(0, \sigma_{\ddot{\psi}}^{2}\right).
\end{align}
$$

The influence of the noise processes on the process model can be expressed in state-space form as:

$$
\begin{align}
x_{k+1} = x_{k} + 
\begin{bmatrix}
\frac{v_{k}}{\dot{\psi}_{k}} \left(\sin\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) - \sin\left(\psi_{k}\right)\right) \\
\frac{v_{k}}{\dot{\psi}_{k}} \left(-\cos\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) + \cos\left(\psi_{k}\right)\right) \\
0 \\ 
\dot{\psi}_{k} \Delta t\\
0
\end{bmatrix} +
\begin{bmatrix}
\frac{1}{2}\left(\Delta t\right)^{2} \cos\left(\psi_{k}\right)\cdot \nu_{a, k} \\
\frac{1}{2}\left(\Delta t\right)^{2} \sin\left(\psi_{k}\right)\cdot \nu_{a, k} \\
\Delta t \cdot \nu_{a, k} \\
\frac{1}{2}\left(\Delta t\right)^{2} \cdot \nu_{\ddot{\psi}, {k}} \\
\Delta t \cdot \nu_{\ddot{\psi}, k} \\
\end{bmatrix}
\end{align}
$$

and relies on the constant turn-rate assumption of the CTRV model, i.e., that the vehicle travels straight at time $t_{k+1}$ along the hypotenuse of the right-triangle formed in Figure 1 above. Note that this also assumes the effect of the yaw acceleration on the position to be zero, which in the non-zero case will result in the vehicle being on either side of the hypotenuse (since the non-zero yaw acceleration changes the radius of the circle formed by the trajectory the vehicle is travelling on).

With the CTRV model equations covered, we can now move onto the implementation of the Unscented Kalman filter (UKF). As we learned earlier, the UKF does not require us to compute the Jacobian matrix to linearise the non-linear process or measurement models. Instead, we form the representative sigma points sampled from a Gaussian distribution. These points will be then plugged into the non-linear process model equation above in order to estimate a new state mean and covariance using very similar equations to the standard Kalman filter for both the predict and innovation / update steps.

##### Choosing process noise design parameter values

For the CTRV model, we have two design parameters representing process noise: $\sigma_{a}^{2}$ and $\sigma_{\ddot{\phi}}^{2}$. These are the longitudinal acceleration (i.e., linear acceleration) noise, and the yaw acceleration (i.e., angular acceleration) noise variances, respectively. In this part we will briefly cover the [estimation of these parameters](https://en.wikipedia.org/wiki/Kalman_filter#Estimation_of_the_noise_covariances_Qk_and_Rk). 

In order to "tune" these parameters to achieve optimal performance (the "best" covariance estimations), we have to first consider both the object and the scenario in which we are attempting to operate. For example, if we were designing a UKF to track a bicycle, we would want to select these two design parameters' values much differently than if we were designing for a vehicle tracking application. This is because of vehicle behaviour and object kinematics (motion assumptions). Additionally, we would want to select these two design parameters' values based on the driving scenario we are operating in. For example, we might expect a vehicle yaw angle to change much less frequently on a highway with nominal traffic conditions than in an urban environment with heavy traffic, where a vehicle might experience much more frequent sudden changes in speed and direction. Let's start by defining several rules-of-thumb for each parameter.

The linear acceleration noise $\sigma_{a}^{2}$ has the units $\mathrm{m}^{2} / \mathrm{s}^{4}$. If we take the square-root of the linear acceleration noise, i.e., $\sqrt{\sigma_{a}^{2}}$, we end up with a set of units $\mathrm{m} / \mathrm{s}^{2}$, which matches the units of acceleration. This will help us put our noise value into a frame of reference we can interpret using object kinematics. Next, we exploit the process noise model to derive insight about the distribution of the noise values. Since we chose to model the linear acceleration noise as a white-noise, normally-distributed process represented by a zero-mean Gaussian distribution parameterised by the standard deviation $\sigma_{a} = \pm\sqrt{\sigma_{a}^{2}} \mathrm{m}/\mathrm{s}^{2}$, we know that approximately 95% of our noise values are assumed to be within the standard deviation of $2\sigma_{a}$. Therefore, if we select a $\left(\sigma_{a}\right)^{2} = \left(9 \mathrm{m}/{\mathrm{s}^{2}}\right)^{2}$, then we know from our noise process assumption that we have a linear acceleration noise between $2\times \sqrt{\sigma_{a}^{2}} =  \pm 6 \mathrm{m}/{\mathrm{s}^{2}}$ roughly 95% of the time. With this we can choose a respective linear acceleration design parameter $\sigma_{a}^{2}$ based on the expected behaviour of the object in the scenario we are attempting to model. As a rule-of-thumb, this parameter should be given a value in relation to the amount of expected maximum change in linear velocity. For vehicles in highly-dynamic environments with lots of stop-and-go braking / acceleration, such as in urban settings with dense traffic conditions, a choice of $\left(\sigma_{a}\right)^{2} = \left(9 \mathrm{m}/\mathrm{s}^{2}\right)^{2}$ might be sufficient. For less-dynamic, more infrequent stop-and-go braking / acceleration, such an on highways with nominal traffic conditions, a choice of $\left(\sigma_{a}\right)^{2} = \left(3 \mathrm{m}/\mathrm{s}^{2}\right)^{2}$ might be sufficient.

The yaw acceleration noise $\sigma_{\ddot{\psi}}^{2}$ has the units $\mathrm{rad}/\mathrm{s}^{2}$. This gives us an expression for the expected rate-of-change in the object's direction due to noise in the process model. Considering both the environment in which the system will operate — such as in an urban city, or along a highway, and the dynamics of the environment — such as the amount of turns / changes in direction expected, can help one select this yaw acceleration noise value appropriately. In highway driving scenarios with nominal traffic conditions, a choice of yaw acceleration noise $\left(\sigma_{\ddot{\psi}}\right)^{2} = \left(0.1 \mathrm{rad}/\mathrm{s}^{2}\right)^{2}$ might be sufficient. For more-dynamic environments with dense traffic and tight city blocks, a choice of $\left(\sigma_{\ddot{\psi}}\right)^{2} = \left(0.3\mathrm{rad}/\mathrm{s}^{2}\right)^{2}$ might be sufficient.

##### Evaluating design parameter values with Normalised Innovation Squared (NIS) 

Choosing process noise design parameters blindly, or with faulty / incorrect assumptions can lead to poor performance and consistency in the UKF. Therefore, a consistency check called the [Normalised Innovation Squared](https://kalman-filter.com/normalized-innovation-squared/) (NIS) method can be applied to determine if the Kalman filter is consistent with the current measurement residual $\nu\left(k\right) = \left(z_{k+1} - z_{k+1\vert k}\right)$ and the measurement covariance matrix $\mathrm{S}_{k+1\vert k}$. The resulting consistency score $\epsilon_{\nu}$ gives an estimate of the acceptance region in which the normalised difference in the innovation step should be when the Kalman filter is performing as-expected. This score is therefore given as:

$$
\begin{align}
\mathrm{NIS}\left(\nu\left(k\right), \mathrm{S}_{k}\right) &= \epsilon_{\nu} = \left(z_{k+1} - z_{k+1\vert k} \right)^{\top}\cdot \mathrm{S}_{k+1\vert k}^{-1}\cdot \left(z_{k+1} - z_{k+1\vert k}\right).
\end{align}
$$

In order to evaluate the NIS error $\epsilon_{\nu}$, we first make the assumption that $\epsilon_{\nu}$ is sampled from a [chi-square distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution) with dimensions $\mathrm{dim}\left(\nu\left(k\right)\right) = n_{z}$, i.e., the dimension of the measurement vector $z_{k+1}$. Then, a hypothesis test is performed with the following setup:

* $\mathrm{H}_{0}$: The measurement residual $\nu\left(k\right)$ is consistent with the innovation measurement covariance matrix $\mathrm{S}_{k+1\vert k}$ and is accepted if $\epsilon_{\nu} \in \left[r_{1}, r_{2}\right]$.

The acceptance interval is chosen such that the probability $\mathrm{P}$ that $\mathrm{H}_{0}$ is accepted is $\left(1 - \alpha\right)$. The confidence interval $\left[r_{1}, r_{2}\right]$ is calculated using the inverse of the cumulative distribution function (CDF) of the chi-square distribution as follows:

$$
\begin{align}
r_{1} &= \mathrm{F}^{-1}\left(\frac{\alpha}{2}, n_{z}\right), r_{2} = \mathrm{F}^{-1}\left(1 - \frac{\alpha}{2}, n_{z}\right).
\end{align}
$$

One can alternatively use a look-up table for the given measurement vector dimension $n_{x}$ and acceptance region $\left(1-\alpha\right)$.

Additionally, plotting the NIS score for each time-step $k$ may be a helpful indication of consistency. NIS scores that consistently place near the confidence threshold $\alpha$ (e.g., $\alpha = 0.95$) tend to indicate that the selected design parameters are performing well. NIS score values that are _well below_ the $\alpha$ threshold indicate that the design parameters _overestimate_ the uncertainty, and conversely NIS score values that are consistently _well above_ the $\alpha$ threshold indicate that the design parameters _underestimate_ the uncertainty.

In conclusion, the Normalised Innovation Squared (NIS) test can help one evaluate the consistency of the innovation measurement covariance matrix $\mathrm{S}_{k+1\vert k}$. Since this matrix directly influences the Kalman gain $\mathrm{K}$ and therefore the overall efficiency of the Kalman filter, taking the time to evaluate this metric is generally a great idea. Fun fact before we move on: The NIS test comes from the [Normalised Estimation Error Squared](https://kalman-filter.com/normalized-estimation-error-squared/) (NEES) test, but is practical in settings where ground-truth data is not provided. While the NEES test provides deeper insight in "truth model simulation" settings, the NIS test can be used in online real-world applications which do not have access to the true state vector.

#### Alternatives

Before we close this discussion on the Constant Turn Rate and Velocity Magnitude (CTRV) model, we want to mention that there are several other common motion models one may consider. Those are:

* **Constant Turn Rate and Acceleration (CTRA)**;
* **Constant Steering Angle and Velocity (CSAV)**;
* **Constant Curvature and Acceleration (CCA)**.

Each model makes a different assumption about the vehicle's motion, and therefore choosing a model is quite an important step. For the remainder of this notebook, we make explicit reference to the CTRV model when designing our Unscented Kalman filter in C++.

## 2. Programming Task

In this section we combine our knowledge of the Unscented Kalman filter (UKF) and the Constant Turn Rate and Velocity Magnitude (CTRV) model with our C++ programming skills to implement the filter for vehicle tracking.

Here we use the [Eigen](https://en.wikipedia.org/wiki/Eigen_\(C%2B%2B_library\)) C++ matrix manipulation library to represent and perform operations on both the vectors and matrices we described earlier. 

### 2.1. Unscented Kalman Filter in C++

In [1]:
// From J. Moran's `ukf.h`

In [2]:
/* ----------------------------------------------------------------------------
 * Lesson "2.5: Unscented Kalman Filters"
 * Authors     : Dominik Nuss, Andrei Vatavu.
 *
 * Modified by : Jonathan L. Moran (jonathan.moran107@gmail.com)
 *
 * Purpose of this file: Header file for the Unscented Kalman Filter (UKF).
 * ----------------------------------------------------------------------------
 */

#include "include/eigen/Eigen/Dense"
#include <random>
#include <cmath>
#include <iostream>


/* The Unscented Kalman Filter (UKF) class.
 */
class UKF {
 public:
  UKF() {};
  virtual ~UKF() {};
  // Initalises the UKF instance
  void InitFilter();
  // Computes the sigma points from the state vector / covariance matrix
  void GenerateSigmaPoints(
      Eigen::MatrixXd* Xsig_out
  );
  // Computes the sigma points from the augemnted state / covariance
  void AugmentedSigmaPoints(
      Eigen::MatrixXd* Xsig_out
  );
  // Evaluates the state prediction / state transition function 
  void SigmaPointPrediction(
      Eigen::MatrixXd* Xsig_out
  );
  // Normalises heading angle of state estimation to a value in range [-pi, pi]
  Eigen::VectorXd NormaliseHeading(
      Eigen::VectorXd Xsig_pred_diff
  );
  // Performs the prediction of the state estimation and covariance matrix
  void PredictMeanAndCovariance(
      Eigen::VectorXd* x_pred, 
      Eigen::MatrixXd* P_pred
  );
  // Predicts the radar measurement vector and covariance in innovation step
  void PredictRadarMeasurement(
      Eigen::VectorXd* z_out, 
      Eigen::MatrixXd* S_out
  );
  void UpdateState(
      Eigen::VectorXd* x_out, 
      Eigen::MatrixXd* P_out
  );
};

#### 2.1.1. Generating Sigma Points

In [3]:
// From J. Moran's `ukf.cc`

In [4]:
/* Constructs the sigma point matrix.
 *
 * Implements the sigma point estimation for use in the prediction step.
 * The UKF performs an unscented transformation which approximates a normal
 * distribution from the non-linear posterior state estimation.
 *
 * Here the sigma points are determined with respect to the dimension of the
 * state vector and a spreading factor $\lambda$. This design parameter
 * governs how "close" a sigma point is to the mean $x_{k\vert k}$, i.e.,
 * the posterior state estimation from the previous time-step. The posterior
 * covariance matrix $\mathrm{P}_{k\vert k}$ is also used in the calculation
 * of the sigma points. The output matrix $\mathcal{x}_{k\vert k} stores
 * the resulting sigma points (`Xsig_out` in the function).
 * 
 * @param  Xsig_out   Matrix to store resulting sigma points.
 */
void UKF::GenerateSigmaPoints(
    Eigen::MatrixXd* Xsig_out
) {
  // Set the state dimension
  int n_x = 5;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_x + 1; 
  // Define the spreading parameter
  double lambda = 3 - n_x;
  // Set the state vector values
  // Here, this is assumed to be the mean of the posterior state estimation
  Eigen::VectorXd x(n_x);
  x << 5.7441,
       1.3800,
       2.2049,
       0.5015,
       0.3528;
  // Set the covariance matrix values
  // Here, this is assumed to be the covariance of posterior state estimation
  Eigen::MatrixXd P(n_x, n_x);
  P << 0.0043,   -0.0013,    0.0030,   -0.0022,   -0.0020,
      -0.0013,    0.0077,    0.0011,    0.0071,    0.0060,
       0.0030,    0.0011,    0.0054,    0.0007,    0.0008,
      -0.0022,    0.0071,    0.0007,    0.0098,    0.0100,
      -0.0020,    0.0060,    0.0008,    0.0100,    0.0123;
  // Create the sigma point matrix
  Eigen::MatrixXd Xsig(n_x, n_sigma_points);
  // Calculate square-root of matrix `P`
  // `A` is the lower-triangular matrix of the Cholesky decomposition
  Eigen::MatrixXd A = P.llt().matrixL();
  /*** Calculate the set of sigma points ***/
  // Set the first column to the mean of the posterior state estimation
  Xsig.col(0) = x;
  // Compute the square-root term for the sigma point vector
  // The square-root of the spreading term
  double spreading_factor = std::sqrt(lambda + n_x);
  // Loop through the columns of `A` to compute columns of `Xsig`
  for (int i = 0; i < n_x; i++) {
    // First, update the lower column terms
    // Note the array indexing of `A` starting at 0
    Xsig.col(i + 1) = x + spreading_factor * A.col(i);
    // Then, update the upper column terms
    Xsig.col(i + n_x + 1) = x - spreading_factor * A.col(i);
  }
  // Print the resulting matrix
  // std::cout << "Xsig = " << "\n" << Xsig << "\n";
  // Update the input pointer to the output result
  *Xsig_out = Xsig;
}

##### Testing the generate sigma points function

In [5]:
// From J. Moran's `5_tests.cc`

In [6]:
/* Evaluates the result of the `GenerateSigmaPoints` function.
 *
 * The actual sigma points produced by the function are compared to the
 * expected matrix of sigma point values by computing the Frobenius L2
 * norm (matrix norm). If the resulting L2 norm value is less than the
 * `epsilon` threshold, the two matrices are said to be roughly equal.
 */
void test_generate_sigma_points() {
  // Create the Unscented Kalman Filter (UKF) instance
  UKF ukf;
  // Instantiate the sigma point output matrix 
  Eigen::MatrixXd Xsig(5, 11);
  // Generate the sigma points and write them to the output matrix
  ukf.GenerateSigmaPoints(&Xsig);
  // Print the resulting values
  std::cout << "Xsig = " << "\n" << Xsig << "\n";
  // Perform L2 norm to compare the two matrices
  Eigen::MatrixXd Xsig_expected(5, 11);
  Xsig_expected <<
    5.7441,  5.85768,   5.7441,   5.7441,   5.7441,   5.7441,  5.63052,   5.7441,   5.7441,   5.7441,   5.7441,
    1.38,  1.34566,  1.52806,     1.38,     1.38,     1.38,  1.41434,  1.23194,     1.38,     1.38,     1.38,
    2.2049,  2.28414,  2.24557,  2.29582,   2.2049,   2.2049,  2.12566,  2.16423,  2.11398,   2.2049,   2.2049,
    0.5015,  0.44339, 0.631886, 0.516923, 0.595227,   0.5015,  0.55961, 0.371114, 0.486077, 0.407773,   0.5015,
    0.3528, 0.299973, 0.462123, 0.376339,  0.48417, 0.418721, 0.405627, 0.243477, 0.329261,  0.22143, 0.286879;
  double epsilon = 0.001;
  std::cout << "Result matches expected by amount `epsilon = " << epsilon << "`";
  std::cout << ": " << std::boolalpha << Xsig.isApprox(Xsig_expected, epsilon) << "\n";
}

In [7]:
// Exercise 2.5.1: Generating Sigma Points
test_generate_sigma_points();

Xsig = 
  5.7441  5.85768   5.7441   5.7441   5.7441   5.7441  5.63052   5.7441   5.7441   5.7441   5.7441
    1.38  1.34566  1.52806     1.38     1.38     1.38  1.41434  1.23194     1.38     1.38     1.38
  2.2049  2.28414  2.24557  2.29582   2.2049   2.2049  2.12566  2.16423  2.11398   2.2049   2.2049
  0.5015  0.44339 0.631886 0.516923 0.595227   0.5015  0.55961 0.371114 0.486077 0.407773   0.5015
  0.3528 0.299973 0.462123 0.376339  0.48417 0.418721 0.405627 0.243477 0.329261  0.22143 0.286879
Result matches expected by amount `epsilon = 0.001`: true


#### 2.1.2. UKF Augmentation

In [8]:
// From J. Moran's `ukf.cc`

In [9]:
/* Constructs the augmented sigma point matrix. 
 *
 * Implements the sigma point estimation for the process noise $\nu_{k}$
 * used in the prediction step of the Unscented Kalman Filter.
 * 
 * Here the augmented sigma points are determined with respect to the dimension
 * of the augmented state vector $x_{a, k}$ and augmented process noise
 * covariance matrix $\mathrm{P}_{a, k\vert k}$. The mean of the process noise
 * is assumed to be zero. The augmented sigma points are computed with a design
 * parameter $\lambda$ which governs how "close" an augmented sigma point is to
 * the mean $x_{a, k\vert k}$.
 *
 * @param  Xsig_out   Matrix to store resulting augmented sigma points. 
 */
void UKF::AugmentedSigmaPoints(
    Eigen::MatrixXd* Xsig_out
) {
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Process noise standard deviation of longitudinal acceleration (m/s^2)
  double std_a = 0.2;
  // Process noise standard deviation of yaw acceleration (rad/s^2)
  double std_yawdd = 0.2;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Define the independent noise processes
  // Here, both are zero-mean white-noise processes
  std::normal_distribution<double> nu_a(0.0, std_a);
  std::normal_distribution<double> nu_yawdd(0.0, std_yawdd);
  // Define the noise processes vector and set its values
  std::default_random_engine rand_gen;
  Eigen::VectorXd nu_k(n_a, 1);
  nu_k << nu_a(rand_gen),
          nu_yawdd(rand_gen);
  // Set the state vector values
  // Here, this is assumed to be the mean of the posterior state estimation
  Eigen::VectorXd x(n_x);
  x << 5.7441,
       1.3800,
       2.2049,
       0.5015,
       0.3528;
  // Set the covariance matrix values
  // Here, this is assumed to be the covariance of posterior state estimation
  Eigen::MatrixXd P(n_x, n_x);
  P << 0.0043,   -0.0013,    0.0030,   -0.0022,   -0.0020,
      -0.0013,    0.0077,    0.0011,    0.0071,    0.0060,
       0.0030,    0.0011,    0.0054,    0.0007,    0.0008,
      -0.0022,    0.0071,    0.0007,    0.0098,    0.0100,
      -0.0020,    0.0060,    0.0008,    0.0100,    0.0123;
  // Instantiate the augmented mean vector
  Eigen::VectorXd x_aug(n_aug);
  // Instantiate the augmented state covariance matrix
  Eigen::MatrixXd P_aug(n_aug, n_aug);
  // Instantiate the augmented sigma point matrix
  Eigen::MatrixXd Xsig_aug(n_aug, n_sigma_points);
  // Compute the values of the augmented mean state vector
  // Set the first `n_x` values to be the state vector 
  x_aug.head(n_x) = x;
  // Set the last values to be the mean of the noise processes
  x_aug.row(5) << nu_a.mean();
  x_aug.row(6) << nu_yawdd.mean(); 
  // Compute the values of the augmented covariance matrix
  Eigen::MatrixXd Q(n_a, n_a);
  Q << std_a * std_a, 0.0,
       0.0, std_yawdd * std_yawdd;
  P_aug.fill(0.0);
  P_aug.topLeftCorner(n_x, n_x) = P;
  P_aug.bottomRightCorner(n_a, n_a) = Q;
  // Calcualte the square-root of the augmented covariance matrix
  // `A` is the lower-triangular matrix of the Cholesky decomposition
  Eigen::MatrixXd A_aug = P_aug.llt().matrixL();
  /*** Calculate the set of augmented sigma points ***/
  // Set the first column as mean of augmented posterior state estimation
  Xsig_aug.col(0) = x_aug;
  // Compute the square-root term for the augmented sigma point vector
  // The square-root of the spreading term
  double spreading_factor = std::sqrt(lambda + n_aug);
  // Loop through the columns of `A` to compute columns of `Xsig_aug`
  for (int i = 0; i < n_aug; i++) {
    // First, update the lower column terms
    // Note the array indexing of `A` starting at 0
    Xsig_aug.col(i + 1) = x_aug + spreading_factor * A_aug.col(i);
    // Then, update the upper column terms
    Xsig_aug.col(i + n_aug + 1) = x_aug - spreading_factor * A_aug.col(i);
  }
  // Print the resulting augmented sigma point matrix
  // std::cout << "Xsig_aug = " << "\n" << Xsig_aug << "\n";
  // Update the input pointer to the output result
  *Xsig_out = Xsig_aug; 
}

##### Testing the sigma point augmentation

In [10]:
// From J. Moran's `5_tests.cc`

In [11]:
/* Evaluates the result of the `AugmentedSigmaPoints` function.
 *
 * The actual augmented sigma points produced by the function are compared
 * to the expected matrix of augmented sigma point values by computing the
 * Frobenius L2 norm (matrix norm). If the resulting L2 norm value is less
 * than the `epsilon` threshold, the two matrices are said to be roughly
 * equal.
 */
void test_augmented_sigma_points() {
  // Create the Unscented Kalman Filter (UKF) instance
  UKF ukf;
  // Instantiate the augmented sigma point matrix
  // Note: We assume the dimension of the augmented state vector to be `7`,
  // and the number of augmented sigma points generated to be `15`
  Eigen::MatrixXd Xsig_aug(7, 15);
  // Generate the augmented sigma points and write them to the output matrix
  ukf.AugmentedSigmaPoints(&Xsig_aug);
  // Print the resulting values
  std::cout << "Xsig_aug = " << "\n" << Xsig_aug << "\n";
  // Perform the L2 norm to compare the two matrices
  Eigen::MatrixXd Xsig_aug_expected(7, 15);
  Xsig_aug_expected <<
  5.7441,  5.85768,   5.7441,   5.7441,   5.7441,   5.7441,   5.7441,   5.7441,  5.63052,   5.7441,   5.7441,   5.7441,   5.7441,   5.7441,   5.7441,
    1.38,  1.34566,  1.52806,     1.38,     1.38,     1.38,     1.38,     1.38,  1.41434,  1.23194,     1.38,     1.38,     1.38,     1.38,     1.38,
  2.2049,  2.28414,  2.24557,  2.29582,   2.2049,   2.2049,   2.2049,   2.2049,  2.12566,  2.16423,  2.11398,   2.2049,   2.2049,   2.2049,   2.2049,
  0.5015,  0.44339, 0.631886, 0.516923, 0.595227,   0.5015,   0.5015,   0.5015,  0.55961, 0.371114, 0.486077, 0.407773,   0.5015,   0.5015,   0.5015,
  0.3528, 0.299973, 0.462123, 0.376339,  0.48417, 0.418721,   0.3528,   0.3528, 0.405627, 0.243477, 0.329261,  0.22143, 0.286879,   0.3528,   0.3528,
       0,        0,        0,        0,        0,        0,  0.34641,        0,        0,        0,        0,        0,        0, -0.34641,        0,
       0,        0,        0,        0,        0,        0,        0,  0.34641,        0,        0,        0,        0,        0,        0, -0.34641;
  // Precision (i.e., max allowed magnitude of the two matrices' L2 distance)
  double epsilon = 0.001;
  std::cout << "Result matches expected by amount `epsilon = " << epsilon << "`";
  std::cout << ": " << std::boolalpha << Xsig_aug.isApprox(Xsig_aug_expected, epsilon) << "\n"; 
}

In [12]:
// Exercise 2.5.2: Generating Augmented Sigma Points
test_augmented_sigma_points();

Xsig_aug = 
  5.7441  5.85768   5.7441   5.7441   5.7441   5.7441   5.7441   5.7441  5.63052   5.7441   5.7441   5.7441   5.7441   5.7441   5.7441
    1.38  1.34566  1.52806     1.38     1.38     1.38     1.38     1.38  1.41434  1.23194     1.38     1.38     1.38     1.38     1.38
  2.2049  2.28414  2.24557  2.29582   2.2049   2.2049   2.2049   2.2049  2.12566  2.16423  2.11398   2.2049   2.2049   2.2049   2.2049
  0.5015  0.44339 0.631886 0.516923 0.595227   0.5015   0.5015   0.5015  0.55961 0.371114 0.486077 0.407773   0.5015   0.5015   0.5015
  0.3528 0.299973 0.462123 0.376339  0.48417 0.418721   0.3528   0.3528 0.405627 0.243477 0.329261  0.22143 0.286879   0.3528   0.3528
       0        0        0        0        0        0  0.34641        0        0        0        0        0        0 -0.34641        0
       0        0        0        0        0        0        0  0.34641        0        0        0        0        0        0 -0.34641
Result matches expected by amount `epsilon 

#### 2.1.3. Sigma Point Prediction

In [13]:
// From J. Moran's `ukf.cc`

In [14]:
/* Constructs the predicted state estimation from the augmented sigma points.
 *
 * Implements the CTRV model (i.e., Constant Turn Rate and Velocity Magnitude).
 * Here, the augmented sigma points are used to evaluate the non-linear process
 * model function $\mathcal{f}$ w.r.t. $\Delta t$.
 * 
 * The resulting point predictions are written to the right-column of the
 * predicted state estimation matrix (ensuring divide-by-zero does not occur).
 * 
 * @param  Xsig_out   Matrix to store the resulting sigma point predictions.
 */
void UKF::SigmaPointPrediction(
    Eigen::MatrixXd* Xsig_out
) {
  /*** Set the state variables (values should match across functions) ***/
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Get the augmented sigma point matrix
  Eigen::MatrixXd Xsig_aug(n_aug, n_sigma_points);
  AugmentedSigmaPoints(&Xsig_aug);
  /*** Compute the predicted sigma point matrix ***/
  // Instantiate the predicted sigma point matrix
  Eigen::MatrixXd Xsig_pred(n_x, n_sigma_points);
  // Define the delta-time variable (s)
  double delta_t = 0.1;
  // Write the predicted sigma points into right-column of the output matrix
  for (int i = 0; i < n_sigma_points; i++) {
    // Get the state vector values from the augmented sigma point matrix
    Eigen::VectorXd x_k(n_x, 1);
    x_k << Xsig_aug.col(i).head(n_x);
    // Get the mean-values of the noise processes
    Eigen::VectorXd nu_mean_k(n_a, 1);
    // i.e., the last `n_a` values in state vector
    nu_mean_k << Xsig_aug.col(i).tail(n_a);
    // Predict the sigma point by forming the process model in state-space
    // The vector for the state transition equation 
    Eigen::VectorXd Fx_k(n_x, 1);
    // The vector of the process noise values evaluated w.r.t. time
    Eigen::VectorXd nu_k(n_x, 1);
    // Get the variables w.r.t. this sigma point
    double v_k = x_k(2);
    double yaw_k = x_k(3);
    double yawd_k = x_k(4);
    // Avoid a divide-by-zero for $\dot{\psi}$ (the yaw angle rate-of-change)
    if (yawd_k < 0.0001) {
      // Compute the state-space form of the process model
      Fx_k << 
        v_k * std::cos(yaw_k) * delta_t,
        v_k * std::sin(yaw_k) * delta_t,
        0,
        yawd_k * delta_t,
        0;
      // Compute the contribution of the process noise
      nu_k <<
        0.5 * std::pow(delta_t, 2) * std::cos(yaw_k) * nu_mean_k(0),
        0.5 * std::pow(delta_t, 2) * std::sin(yaw_k) * nu_mean_k(0),
        delta_t * nu_mean_k(0),
        0.5 * std::pow(delta_t, 2) * nu_mean_k(1),
        delta_t * nu_mean_k(1);
      // Store the sigma point prediction into the predicted state matrix
      Xsig_pred.col(i) << x_k + Fx_k + nu_k;
      continue;
    }
    // Compute the state-space form of the process model
    Fx_k <<
      (v_k / yawd_k) * (std::sin(yaw_k + yawd_k * delta_t) - std::sin(yaw_k)),
      (v_k / yawd_k) * (-std::cos(yaw_k + yawd_k * delta_t) + std::cos(yaw_k)),
      0,
      yawd_k * delta_t,
      0;
    // Compute the contribution of the process noise
    nu_k << 
      0.5 * std::pow(delta_t, 2) * std::cos(yaw_k) * nu_mean_k(0),
      0.5 * std::pow(delta_t, 2) * std::sin(yaw_k) * nu_mean_k(0),
      delta_t * nu_mean_k(0),
      0.5 * std::pow(delta_t, 2) * nu_mean_k(1),
      delta_t * nu_mean_k(1);
    // Store the sigma point prediction into the predicted state matrix
    Xsig_pred.col(i) << x_k + Fx_k + nu_k;
  }
  // Print the resulting predicted sigma point matrix
  // std::cout << "Xsig_pred = " << "\n" << Xsig_pred << "\n";
  // Update the input pointer to the output result
  *Xsig_out = Xsig_pred;
}

##### Testing the sigma point prediction

In [15]:
// From J. Moran's `5_tests.cc`

In [16]:
/* Evaluates the result of the `SigmaPointPrediction` function.
 *
 * The state transition function of the CTRV model is evaluated using the
 * augmented sigma points computed with the `AugmentedSigmaPoints` function. 
 * The state transition function is given in state-space form and is defined
 * with respect to the $\Delta t$ parameter. The output of this function, i.e.,
 * the resulting point predictions, are written to the predicted state
 * estimation matrix `Xsig_out`. The values of this output matrix are compared
 * to the expected values defined here using the Frobenius L2 norm. If the
 * resulting L2 norm value is less than the `epsilon` threshold, the two
 * matrices are said to be roughly equal.
 */
void test_sigma_point_prediction() {
  // Create the Unscented Kalman Filter (UKF) instance
  UKF ukf;
  // Instantiate the augmented sigma point matrix
  // Assumed to be of dimensions (`n_aug`, `n_sigma_points`) which match the
  // values set within the `AugmentSigmaPoints` function
  Eigen::MatrixXd Xsig_aug(7, 15);
  // Generate the augmented sigma points and write them to the output matrix
  ukf.AugmentedSigmaPoints(&Xsig_aug);
  // Instantiate the output predicted sigma point matrix
  // Assumed to be of dimensions (`n_x`, `n_sigma_points`) which match the
  // values set within the `SigmaPointPrediction` function 
  Eigen::MatrixXd Xsig_pred(5, 15);
  // Compute the output matrix (the predicted state matrix)
  ukf.SigmaPointPrediction(&Xsig_pred);
  // Print the resulting values
  std::cout << "Xsig_pred = " << Xsig_pred << "\n";
  // Perform the L2 norm to compare the two matrices
  Eigen::MatrixXd Xsig_pred_expected(5, 15);
  Xsig_pred_expected <<
    5.93553, 6.06251,  5.92217,  5.9415,   5.92361,  5.93516,  5.93705, 5.93553,  5.80832,  5.94481,  5.92935,  5.94553,  5.93589,  5.93401, 5.93553,
    1.48939, 1.44673,  1.66484,  1.49719,  1.508,    1.49001,  1.49022, 1.48939,  1.5308,   1.31287,  1.48182,  1.46967,  1.48876,  1.48855, 1.48939,
    2.2049,  2.28414,  2.24557,  2.29582,  2.2049,   2.2049,   2.23954, 2.2049,   2.12566,  2.16423,  2.11398,  2.2049,   2.2049,   2.17026, 2.2049,
    0.53678, 0.473387, 0.678098, 0.554557, 0.643644, 0.543372, 0.53678, 0.538512, 0.600173, 0.395462, 0.519003, 0.429916, 0.530188, 0.53678, 0.535048,
    0.3528,  0.299973, 0.462123, 0.376339, 0.48417,  0.418721, 0.3528,  0.387441, 0.405627, 0.243477, 0.329261, 0.22143,  0.286879, 0.3528, 0.318159;
  // Precision (i.e., max allowed magnitude of the two matrices' L2 distance)
  double epsilon = 0.001;
  std::cout << "Result matches expected by amount `epsilon = " << epsilon << "`";
  std::cout << ": " << std::boolalpha << Xsig_pred.isApprox(Xsig_pred_expected, epsilon) << "\n";
}

In [17]:
// Exercise 2.5.3: Prediction Step with Sigma Point
test_sigma_point_prediction();

Xsig_pred =  5.93553   6.0625  5.92217   5.9415  5.92361  5.93516  5.93705  5.93553  5.80833  5.94481  5.92935  5.94553  5.93589  5.93401  5.93553
 1.48939  1.44673  1.66483  1.49719    1.508  1.49001  1.49022  1.48939   1.5308  1.31288  1.48182  1.46967  1.48876  1.48855  1.48939
  2.2049  2.28414  2.24557  2.29582   2.2049   2.2049  2.23954   2.2049  2.12566  2.16423  2.11398   2.2049   2.2049  2.17026   2.2049
 0.53678 0.473388 0.678099 0.554557 0.643644 0.543372  0.53678 0.538512 0.600172 0.395461 0.519003 0.429916 0.530188  0.53678 0.535048
  0.3528 0.299973 0.462123 0.376339  0.48417 0.418721   0.3528 0.387441 0.405627 0.243477 0.329261  0.22143 0.286879   0.3528 0.318159
Result matches expected by amount `epsilon = 0.001`: true


#### 2.1.4. Prediction Step — Mean and Covariance

In [18]:
// From J. Moran's `ukf.cc`

In [19]:
namespace SigmaPoints {
/* Normalises the heading angle of the predicted state estimation.
 *
 * The input vector is assumed to be the difference between the predicted and
 * previous state estimation vector, s.t. the fourth row-wise element of the
 * input vector is the heading angle to normalise in range [-pi, pi].
 *
 * @param    x_diff   Vector of difference values between state estimates.
 * @returns  Normalised vector of the difference between state estimates. 
 */
Eigen::VectorXd NormaliseHeading(
  Eigen::VectorXd x_diff
) {
  while (x_diff(3) < -M_PI) {
    x_diff(3) += 2.0 * M_PI;
  }
  while (x_diff(3) > M_PI) {
    x_diff(3) -= 2.0 * M_PI;
  }
  return x_diff;
}
}  // namespace SigmaPoints


namespace Radar {
/* Normalises the heading angle of the radar measurement state estimatation.
 *
 * The input vector is assumed to be the radar measurement vector s.t. the
 * second row-wise element of the input vector is the heading angle to
 * normalise in range [0, pi].
 *
 * @param    z_diff   Vector of difference values between radar measurements.
 * @returns  Normalised vector of the difference in radar measurements.
 */
Eigen::VectorXd NormaliseHeading(
  Eigen::VectorXd z_diff
) {
  while (z_diff(1) < -M_PI) {
    z_diff(1) += 2.0 * M_PI;
  }
  while (z_diff(1) > M_PI) {
    z_diff(1) -= 2.0 * M_PI;
  }
  return z_diff;
}
} // namespace Radar

In [20]:
// From J. Moran's `ukf.cc`

In [21]:
/* Constructs the predicted the mean state estimation and covariance matrix.
 *
 * The mean state vector and covariance matrix are predicted into the next
 * time-step using the sigma point prediction matrix from the previous step.
 * The predicted values are computed w.r.t. a weight vector which is defined
 * in terms of the spreading parameter $\lambda$. The weights are applied to
 * "undo" the effect of spreading on the covariance and mean state prediction.
 * 
 * @param  x_out  Vector to store predicted mean state estimation. 
 * @param  P_out  Matrix to store predicted covariance.
 */
void UKF::PredictMeanAndCovariance(
    Eigen::VectorXd* x_out, 
    Eigen::MatrixXd* P_out
) {
  /*** Set the state variables (values should match across functions) ***/
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Get the augmented sigma point matrix
  Eigen::MatrixXd Xsig_aug(n_aug, n_sigma_points);
  AugmentedSigmaPoints(&Xsig_aug);
  /*** Compute the predicted sigma point matrix ***/
  // Instantiate the predicted sigma point matrix
  // NOTE: if running unit test in `5_tests.cc`, the values of the matrix
  // must be set manually using the provided definition 
  Eigen::MatrixXd Xsig_pred(n_x, n_sigma_points);
  // Define the delta-time variable (s)
  double delta_t = 0.1;
  // Get the predicted sigma points
  //SigmaPointPrediction(&Xsig_pred);
  Xsig_pred <<
     5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
       1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
      2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
     0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
      0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  /*** Compute the predicted mean state and covariance matrix ***/
  // Instantiate the weight vector
  Eigen::VectorXd w(n_sigma_points, 1);
  // Initialise the predicted mean state vector
  Eigen::VectorXd x(n_x);
  x.fill(0.0);
  // Initialise the predicted covariance matrix
  Eigen::MatrixXd P(n_x, n_x);
  P.fill(0.0);
  // Set the weight vector values
  // Computing the first value of the weight value
  w(0) = lambda / (lambda + n_aug);
  // Computing the rest of the weight values
  for (int i = 1; i < n_sigma_points; i++) {
    w(i) = 1.0 / (2.0 * (lambda + n_aug));
  }
  // Perform the mean state estimation vector prediction
  for (int i = 0; i < n_sigma_points; i++) {
    // Compute the predicted mean state
    x += w(i) * Xsig_pred.col(i);
  }
  // Perform the covariance matrix prediction
  for (int i = 0; i < n_sigma_points; i++) {
    // Compute the difference between the state estimations
    // Then, normalise the resulting heading angle to range [-pi, pi]
    Eigen::VectorXd Xsig_pred_diff = (
      SigmaPoints::NormaliseHeading(Xsig_pred.col(i) - x)
    );
    // Compute the predicted covariance matrix
    P += w(i) * Xsig_pred_diff * Xsig_pred_diff.transpose();
  }
  // Print the resulting mean state and covariance matrix predictions
  std::cout << "Predicted state" << "\n";
  std::cout << x << "\n";
  std::cout << "Predicted covariance matrix" << "\n";
  std::cout << P << "\n";
  // Update the input pointers to the output results
  *x_out = x;
  *P_out = P;
}

##### Testing the mean and covariance prediction

In [22]:
// From J. Moran's `5_tests.cc`

In [23]:
/* Evalautes the result of the `PredictMeanAndCovariance` function.
 * 
 * The mean state estimation and covariance matrix are predicted into the next
 * time-step using the predict step equations of the Unscented Kalman Filter.
 * The prediction relies on the previous sigma point predictions from the
 * `SigmaPointPrediction` function. The resulting heading angle of the difference
 * vector between the previous and the predicted state estimation is normalised
 * to a range [-pi, pi] corresponding to the expected values for the vehicle.
 */
void test_predict_mean_and_covariance() {
  // Create the Unscented Kalman Filter (UKF) isntance
  UKF ukf;
  // Instantiate the predicted state estimation vector
  // Assumed to be of dimensions (`n_x`, 1) which match the
  // values set within the `PredictMeanAndCovariance` function 
  Eigen::VectorXd x(5, 1);
  // Instantiate the predicted covariance matrix
  // Assumed to be of dimensions (`n_x`, `n_x`) which match the
  // values set within the `PredictMeanAndCovariance` function
  Eigen::MatrixXd P(5, 5);
  // Compute the outputs (the predicted state estimation and covariance matrix)
  ukf.PredictMeanAndCovariance(
      &x,
      &P 
  );
  // Printing the resulting values
  std::cout << "x = " << "\n" << x << "\n";
  std::cout << "P = " << "\n" << P << "\n";
  // Perform the L2 norm to compare the output values
  // NOTE: these expected values hold only for the hard-coded `Xsig_pred`
  // matrix whose values are given in the Udacity VM workspace as follows:
  // Xsig_pred <<
  //        5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
  //          1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
  //         2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
  //        0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
  //         0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  Eigen::VectorXd x_expected(5, 1);
  x_expected <<
    5.93637,
    1.49035,
    2.20528,
    0.536853,
    0.353577;
  Eigen::MatrixXd P_expected(5, 5);
  P_expected <<
    0.00543425, -0.0024053,  0.00341576, -0.00348196, -0.00299378,
   -0.0024053,   0.010845,   0.0014923,   0.00980182,  0.00791091,
    0.00341576,  0.0014923,  0.00580129,  0.000778632, 0.000792973,
   -0.00348196,  0.00980182, 0.000778632, 0.0119238,   0.0112491,
   -0.00299378,  0.00791091, 0.000792973, 0.0112491,   0.0126972;
  // Precision (i.e., max allowed magnitude of the outputs' L2 difference)
  // NOTE: see above caveat (test works only when `Xsig_pred` is set manually) 
  double epsilon = 0.001;
  std::cout << "Result `x` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << x.isApprox(x_expected, epsilon) << "\n";
  std::cout << "Result `P` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << P.isApprox(P_expected, epsilon) << "\n";
}

In [24]:
// Exercise 2.5.4: Prediction Step with Mean and Covariance
test_predict_mean_and_covariance();

Predicted state
 5.93637
 1.49035
 2.20528
0.536853
0.353577
Predicted covariance matrix
 0.00543425  -0.0024053  0.00341576 -0.00348196 -0.00299378
 -0.0024053    0.010845   0.0014923  0.00980182  0.00791091
 0.00341576   0.0014923  0.00580129 0.000778632 0.000792973
-0.00348196  0.00980182 0.000778632   0.0119238   0.0112491
-0.00299378  0.00791091 0.000792973   0.0112491   0.0126972
x = 
 5.93637
 1.49035
 2.20528
0.536853
0.353577
P = 
 0.00543425  -0.0024053  0.00341576 -0.00348196 -0.00299378
 -0.0024053    0.010845   0.0014923  0.00980182  0.00791091
 0.00341576   0.0014923  0.00580129 0.000778632 0.000792973
-0.00348196  0.00980182 0.000778632   0.0119238   0.0112491
-0.00299378  0.00791091 0.000792973   0.0112491   0.0126972
Result `x` matches expected amount by `epsilon = 0.001`: true
Result `P` matches expected amount by `epsilon = 0.001`: true


### 2.1.5. Innovation Step — Radar Measurements

In [25]:
// From J. Moran's `ukf.cc`

In [26]:
/* Constructs the radar measurement mean estimation and covariance matrix.
 * 
 * The radar measurement vector is a three-dimensional vector:
 *   $z_{k + 1\vert k} = [\rho, \phi, \dot{\rho}]$,
 * where $\rho$ is the radial distance (m), $\phi$ is the angle (rad), and
 * $\dot{\rho}$ is the radial velocity (m/s) measured by the radar sensor.
 * 
 * In order to perform the update / innovation step, several "shortcuts" are
 * used. The first involves the "recycling" of the sigma point matrix, which,
 * due to the purely additive (linear) contribution of the radar measurement
 * noise, allows us to skip the computation of new sigma points and the UKF
 * augmentation step all-together. The second "shortcut" used here the hold-out
 * of the measurement noise $\omega_{k+1}$ from the measurement model function.
 * Again, due to the linearity of the noise contribution, we neglect the noise
 * value until the computation of the measurement covariance prediction matrix
 * $\mathrm{R}$ in the innovation / correction step.
 * 
 * The equations for the sigma point-to-radar measurement space are given by:
 *    $\rho = \sqrt{p_{x}^{2} + p_{y}^{2}}$,
 *    $\phi = \arctan(p_{y} / p_{x})$,
 *    $$\begin{align}
 *    \dot{\rho} &= \frac{
 *        p_{x}\cos(\phi)*v + p_{y}\sin(\phi)*v
 *        }{\sqrt{p_{x}^{2} + p_{y}^{2}}} \\
 *    \end{align}$$.
 * 
 * @param  z_out  Vector to store predicted measurement mean state estimation.
 * @param  S_out  Matrix to store predicted measurement covariance.
 */
void UKF::PredictRadarMeasurement(
    Eigen::VectorXd* z_out, 
    Eigen::MatrixXd* S_out
) {
  /*** Set the state variables (values should match across functions) ***/
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Set the measurement dimensions
  // Note: the radar measurement vector is $[\rho, \phi, \dot{\rho}]$
  // i.e., the measured radial distance, angle, and radial velocity
  int n_z = 3;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Set the weight vector values
  Eigen::VectorXd w(n_sigma_points, 1);
  w(0) = lambda / (lambda + n_aug);
  double weight = 1.0 / (2.0 * (lambda + n_aug));
  for (int i = 1; i < n_sigma_points; ++i) {  
    w(i) = weight;
  }
  // Standard deviation of the radar measurement noise for $\rho$ (m)
  double std_radr = 0.3;
  // Standard deviation of the radar measurment noise for $\phi$ (rad)
  double std_radphi = 0.0175;
  // Standard deviation of the radar measurment noise for $\dot{\rho}$ (m/s)
  double std_radrd = 0.1;
  // Define the measurement noise covariance $\mathrm{R}$
  Eigen::MatrixXd R(n_z, n_z);
  // Set the measurement noise covariance matrix values
  R << std_radr * std_radr, 0.0, 0.0,
       0.0, std_radphi * std_radphi, 0.0,
       0.0, 0.0, std_radrd * std_radrd;
  // Get the predicted sigma point matrix
  Eigen::MatrixXd Xsig_pred(n_x, 2 * n_aug + 1);
  //SigmaPointPrediction(&Xsig_pred);
  Xsig_pred <<
     5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
       1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
      2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
     0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
      0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  // Initialise the sigma point matrix in measurement space
  Eigen::MatrixXd Zsig(n_z, n_sigma_points);
  Zsig.fill(0.0);
  // Initialise the mean predicted measurement vector
  Eigen::VectorXd z_pred(n_z);
  z_pred.fill(0.0);
  // Initialise the measurement covariance matrix S
  Eigen::MatrixXd S(n_z, n_z);
  S.fill(0.0);
  // Transform the sigma points into the radar measurement space
  for (int i = 0; i < n_sigma_points; i++) {
    // Get the state vector associated with the sigma point
    Eigen::VectorXd x_i = Xsig_pred.col(i);
    // Store the transformed state vector in measurement space
    Eigen::VectorXd Zsig_i(n_z, 1);
    // Compute the radial distance $\rho$ transformation
    Zsig_i(0) = std::sqrt(std::pow(x_i(0), 2) + std::pow(x_i(1), 2));
    // Compute the angle $\phi$ transformation
    Zsig_i(1) = std::atan(x_i(1) / x_i(0));
    // Compute the radial velocity $\dot{\rho}$ transformation
    Zsig_i(2) = (
      x_i(0) * std::cos(x_i(3)) * x_i(2) + x_i(1) * std::sin(x_i(3)) * x_i(2)
    ) / std::sqrt(std::pow(x_i(0), 2) + std::pow(x_i(1), 2));
    // Update the resulting sigma point in measurment space vector
    Zsig.col(i) = Zsig_i;
  }
  // Calculate the mean predicted measurement vector
  for (int i = 0; i < n_sigma_points; i++) {
    // Compute the `i`th predicteded measurement mean $z_{k+1 \vert k}$
    z_pred += w(i) * Zsig.col(i);
  }
  // Calculate the covariance matrix `S` of the innovation / correction step
  for (int i = 0; i < n_sigma_points; i++) {
    // Compute the difference in measurement mean state estimations
    // Then, normalise the resulting heading angle estimate to range [-pi, pi]
    Eigen::VectorXd Zsig_meas_diff = (
      Radar::NormaliseHeading(Zsig.col(i) - z_pred)
    );
    // Compute the `i`th covariance matrix $S_{k+1\vert k}$ step
    S += w(i) * Zsig_meas_diff * Zsig_meas_diff.transpose();
  }
  // Set the additive contribution of the measurement noise to the vector
  // as defined by the measurement noise covariance matrix $\mathrm{R}$
  S += R;
  // Print the resulting outputs
  std::cout << "z_pred: " << "\n" << z_pred << "\n";
  std::cout << "S: " << "\n" << S << "\n";
  // Update the input pointers to the output results
  *z_out = z_pred;
  *S_out = S;
}

##### Testing the radar measurement prediction

In [27]:
/* Evalautes the result of the `PredictRadarMeasurement` function.
 */
void test_predict_radar_measurement() {
  // Create the Unscented Kalman Filter (UKF) isntance
  UKF ukf;
  // Instantiate the predicted state estimation vector
  // Assumed to be of dimensions (`n_z`, 1) which match the
  // values set within the `PredictRadarMeasurement` function 
  Eigen::VectorXd z_pred(3, 1);
  // Instantiate the predicted covariance matrix
  // Assumed to be of dimensions (`n_z`, `n_z`) which match the
  // values set within the `PredictRadarMeasurement` function
  Eigen::MatrixXd S(3, 3);
  // Compute the outputs (the measurement state estimation / covariance matrix)
  // NOTE: must set the `Xsig_pred` matrix to the values defined below in order
  // to obtain output `z_pred` and `S` with corresponding expected values
  ukf.PredictRadarMeasurement(
      &z_pred,
      &S
  );
  // Printing the resulting values
  std::cout << "z_pred = " << "\n" << z_pred << "\n";
  std::cout << "S = " << "\n" << S << "\n";
  // Perform the L2 norm to compare the output values
  // NOTE: these expected values hold only for the hard-coded `Xsig_pred`
  // matrix whose values are given in the Udacity VM workspace as follows:
  // Xsig_pred <<
  //        5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
  //          1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
  //         2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
  //        0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
  //         0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  Eigen::VectorXd z_pred_expected(3, 1);
  z_pred_expected <<
    6.12155,
    0.245993,
    2.10313;
  Eigen::MatrixXd S_expected(3, 3);
  S_expected <<
    0.0946171,  -0.000139448,  0.00407016,
   -0.000139448, 0.000617548, -0.000770652,
    0.00407016, -0.000770652,  0.0180917;
  // Precision (i.e., max allowed magnitude of the outputs' L2 difference)
  // NOTE: see above caveat (test works only when `Xsig_pred` is set manually) 
  double epsilon = 0.0001;
  std::cout << "Result `z_pred` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << z_pred.isApprox(z_pred_expected, epsilon) << "\n";
  std::cout << "Result `S` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << S.isApprox(S_expected, epsilon) << "\n";
}

In [28]:
// Exercise 2.5.5: Innovation Step with Radar Measurement
test_predict_radar_measurement();

z_pred: 
 6.12155
0.245993
 2.10313
S: 
   0.0946171 -0.000139448   0.00407016
-0.000139448  0.000617548 -0.000770652
  0.00407016 -0.000770652    0.0180917
z_pred = 
 6.12155
0.245993
 2.10313
S = 
   0.0946171 -0.000139448   0.00407016
-0.000139448  0.000617548 -0.000770652
  0.00407016 -0.000770652    0.0180917
Result `z_pred` matches expected amount by `epsilon = 0.0001`: true
Result `S` matches expected amount by `epsilon = 0.0001`: true


### 2.1.6. Innovation Step — Update Mean and Covariance

In [29]:
// From J. Moran's `ukf.cc`

In [30]:
/* Performs the UKF Innovation step to update the state mean and covariance.
 * 
 * The Innovation step of the Unscented Kalman Filter (UKF) differs from
 * the standard Kalman filter (KF) only by a cross-correlation term
 * computed between the state- and measurement space of the sigma points.
 * This term is used in the equation for the Kalman gain given by:
 * $$\begin{align}
 * \mathrm{K}_{k+1\vert k} &= \mathrm{T}_{k+1\vert k} 
 *                            * \mathrm{S}^{\inv}_{k+1\vert k}. 
 * \end{align}$$
 * 
 * The state update equation for the time-step $k+1$ is given by:
 * $$\begin{align}
 * x_{k+1\vert k+1} = x_{k+1\vert k}
 *                    + \mathrm{K}_{k+1\vert k}
 *                    * (z_{k+1} - z_{k+1\vert k}).
 * \end{align}$$
 * 
 * The covariance matrix update equation for time-step $k+1$ is given by:
 * $$\begin{align}
 * \mathrm{P}_{k+1\vert k+1} = \mathrm{P}_{k+1\vert k}
 *                             - \mathrm{K}_{k+1 \ k}
 *                             * \mathrm{S}_{k+1 \vert k}
 *                             * \mathrm{K}^{\top}_{k+1 \vert k}.
 * \end{align}$$
 * 
 * The cross-correlation matrix between the sigma points in state- and
 * measurement-space is given as:
 * $$\begin{align}
 * \mathrm{T}_{k+1\vert k} = 
 * \sum_{i=0}^{2*n_{a}} w_{i} 
 *                      * (\mathcal{x}_{k+1\vert k,i} - x_{k+1\vert k})
 *                      * (\mathcal{z}_{k+1\vert k,i} - z_{k+1\vert k})^{\top}.
 * \end{align}$$
 * 
 * @param  x_out    Vector to store the updated mean state estimate. 
 * @param  P_out    Matrix to store the updated covariance matrix estimate.
 */
void UKF::UpdateState(
    Eigen::VectorXd* x_out,
    Eigen::MatrixXd* P_out
) {
 /*** Set the state variables (values should match across functions) ***/
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Set the measurement dimensions
  // Note: the radar measurement vector is $[\rho, \phi, \dot{\rho}]$
  // i.e., the measured radial distance, angle, and radial velocity
  int n_z = 3;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Set the weight vector values
  Eigen::VectorXd w(n_sigma_points, 1);
  w(0) = lambda / (lambda + n_aug);
  double weight = 1.0 / (2.0 * (lambda + n_aug));
  for (int i = 1; i < n_sigma_points; ++i) {  
    w(i) = weight;
  }
  /*** Compute the predicted sigma point matrix ***/
  // Define the delta-time variable (s)
  double delta_t = 0.1;
  // Instantiate the predicted sigma point matrix
  // NOTE: if running unit test in `5_tests.cc`, the values of the matrix
  // must be set manually using the provided definition
  Eigen::MatrixXd Xsig_pred(n_x, n_sigma_points);
  Xsig_pred <<
     5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
       1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
      2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
     0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
      0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  // Get the predicted sigma points
  //SigmaPointPrediction(&Xsig_pred);
  /*** Compute the predicted mean state and covariance matrix ***/
  // Initialise the predicted mean state vector
  Eigen::VectorXd x(n_x);
  x.fill(0.0);
  // NOTE: if running unit test in `5_tests.cc`, the values of the vector
  // must be set manually using the provided definition
  x <<
     5.93637,
     1.49035,
     2.20528,
    0.536853,
    0.353577;
  // Initialise the predicted covariance matrix
  Eigen::MatrixXd P(n_x, n_x);
  P.fill(0.0);
  // NOTE: if running unit test in `5_tests.cc`, the values of the matrix
  // must be set manually using the provided definition
  P <<
    0.0054342,  -0.002405,  0.0034157, -0.0034819, -0.00299378,
    -0.002405,    0.01084,   0.001492,  0.0098018,  0.00791091,
    0.0034157,   0.001492,  0.0058012, 0.00077863, 0.000792973,
   -0.0034819,  0.0098018, 0.00077863,   0.011923,   0.0112491,
   -0.0029937,  0.0079109, 0.00079297,   0.011249,   0.0126972;
  //PredictMeanAndCovariance(&x, &P);
  /*** Obtain the measurement vector and covariance matrix estimations ***/
  // Instantiate the measurement matrix
  Eigen::MatrixXd Zsig(n_z, n_sigma_points);
  // NOTE: if running unit test in `5_tests.cc`, the values of the matrix
  // must be set manually using the provided definition
  Zsig <<
    6.1190,  6.2334,  6.1531,  6.1283,  6.1143,  6.1190,  6.1221,  6.1190,  6.0079,  6.0883,  6.1125,  6.1248,  6.1190,  6.1188,  6.12057,
   0.24428,  0.2337, 0.27316, 0.24616, 0.24846, 0.24428, 0.24530, 0.24428, 0.25700, 0.21692, 0.24433, 0.24193, 0.24428, 0.24515, 0.245239,
    2.1104,  2.2188,  2.0639,   2.187,  2.0341,  2.1061,  2.1450,  2.1092,  2.0016,   2.129,  2.0346,  2.1651,  2.1145,  2.0786,  2.11295;
  // Instantiate the mean measurement vector
  Eigen::VectorXd z_pred(n_z);
  // NOTE: if running unit test in `5_tests.cc`, the values of the vector
  // must be set manually using the provided definition
  z_pred <<
    6.12155,
     0.245993,
     2.10313;
  // Instantiate the predicted measurement covariance matrix
  Eigen::MatrixXd S(n_z, n_z);
  // NOTE: if running unit test in `5_tests.cc`, the values of the matrix
  // must be set manually using the provided definition
  S <<
      0.0946171, -0.000139448,   0.00407016,
   -0.000139448,  0.000617548, -0.000770652,
     0.00407016, -0.000770652,    0.0180917;
  //PredictRadarMeasurement(&z_pred, &S);
  // Instantiate the incoming radar measurement vector
  Eigen::VectorXd z(n_z);
  // NOTE: if running unit test in `5_tests.cc`, the values of the vector
  // must be set manually using the provided definition
  z <<
     5.9214,   // rho in m
     0.2187,   // phi in rad
     2.0062;   // rho_dot in m/s
  /*** Compute the updated state and covariance (innovation step) ***/
  // Initialise the cross-correlation matrix
  Eigen::MatrixXd Tc(n_x, n_z);
  Tc.fill(0.0);
  // Calculate the cross-correlation matrix
  for (int i = 0; i < n_sigma_points; i++) {
    // Compute the difference in the sigma points in state-space
    Eigen::VectorXd x_diff = SigmaPoints::NormaliseHeading(Xsig_pred.col(i) - x); 
    // Compute the difference in the sigma points in measurement-space
    Eigen::VectorXd z_diff = Radar::NormaliseHeading(Zsig.col(i) - z_pred);
    // Compute the cross-correlation for this sigma point
    Tc += w(i) * x_diff * z_diff.transpose(); 
  }
  // Calculate the Kalman gain matrix `K`
  Eigen::MatrixXd K = Tc * S.inverse();
  // Compute the residual
  // i.e., the difference in predicted and received measurement state
  Eigen::VectorXd z_diff = Radar::NormaliseHeading(z - z_pred);
  // Perform the state mean update
  x += K * z_diff;
  // Perform the covariance matrix update
  P -= K * S * K.transpose();
  // Print the resulting outputs
  std::cout << "Updated state x: " << "\n" << x << "\n";
  std::cout << "Updated state covariance P: " << "\n" << P << "\n";
  // Update the input pointers to the outputs
  *x_out = x;
  *P_out = P;
}

##### Testing the mean and covariance update

In [31]:
// From J. Moran's `5_tests.cc`

In [32]:
/* Evaluates the result of the mean and covariance update.
 *
 * Performs the second and final part of the UKF innovation step.
 * The mean state and covariance matrix are updated w.r.t. the incoming
 * radar measurement using the unscented transformation of the sigma points.
 * Here, we assume a purely additive contribution of measurement noise on the
 * radar measurement model. This allows us to simplify the update step and
 * estimate a noise covariance $\mathrm{R}$ to be added in the update of the
 * covariance matrix $\mathrm{P}$. Also defined relative to our model
 * assumptions is the use of the cross-correlation matrix $\mathrm{T}$ which
 * is the cross-correlation difference in the sigma points in state-space and
 * the sigma points in measurement space. This term is used to modify the
 * standard Kalman filter update equations for gain $\mathrm{K}$ and
 * consequently covariance $\mathrm{P}$ and state $x$.
 */
void test_update_state() {
  // Create the Unscented Kalman Filter (UKF) instance
  UKF ukf;
  /*** Set the state variables (values should match across functions) ***/
  // Set the state dimension
  int n_x = 5;
  // Set the augmented dimension
  // Process noise $\nu_{k}$ has the terms $\nu_{a, k}$, $\nu_{\ddot{psi},k}$
  int n_a = 2;
  // The process noise dimension added to the state vector dimension
  int n_aug = n_x + n_a;
  // Calculate the number of sigma points to compute
  int n_sigma_points = 2 * n_aug + 1;
  // Set the measurement dimensions
  // Note: the radar measurement vector is $[\rho, \phi, \dot{\rho}]$
  // i.e., the measured radial distance, angle, and radial velocity
  int n_z = 3;
  // Define the spreading parameter
  double lambda = 3 - n_aug;
  // Define the delta-time variable (s)
  double delta_t = 0.1;
  // Define the incoming radar measurement vector
  // assumed to be the same in `UpdateState`
  Eigen::VectorXd z(n_z);
  z <<
     5.9214,   // rho in m
     0.2187,   // phi in rad
     2.0062;   // rho_dot in m/s
  // Instantiate the updated state estimation
  Eigen::VectorXd x(n_x, 1);
  // Instantiate the updated covariance matrix
  Eigen::MatrixXd P(n_x, n_x);
  /*** Define the variables needed for the test case ***/
  // NOTE: the following values must be set inside `UpdateState`
  // in order for the expected test case values to hold
  Eigen::MatrixXd Xsig_pred(n_x, n_sigma_points);
  Xsig_pred <<
     5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744,
       1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486,
      2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049,
     0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048,
      0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159;
  //Eigen::VectorXd x(n_x);
  x <<
     5.93637,
     1.49035,
     2.20528,
    0.536853,
    0.353577;
  //Eigen::MatrixXd P(n_x, n_x);
  P <<
    0.0054342,  -0.002405,  0.0034157, -0.0034819, -0.00299378,
    -0.002405,    0.01084,   0.001492,  0.0098018,  0.00791091,
    0.0034157,   0.001492,  0.0058012, 0.00077863, 0.000792973,
   -0.0034819,  0.0098018, 0.00077863,   0.011923,   0.0112491,
   -0.0029937,  0.0079109, 0.00079297,   0.011249,   0.0126972;
  Eigen::MatrixXd Zsig(n_z, n_sigma_points);
  Zsig <<
    6.1190,  6.2334,  6.1531,  6.1283,  6.1143,  6.1190,  6.1221,  6.1190,  6.0079,  6.0883,  6.1125,  6.1248,  6.1190,  6.1188,  6.12057,
   0.24428,  0.2337, 0.27316, 0.24616, 0.24846, 0.24428, 0.24530, 0.24428, 0.25700, 0.21692, 0.24433, 0.24193, 0.24428, 0.24515, 0.245239,
    2.1104,  2.2188,  2.0639,   2.187,  2.0341,  2.1061,  2.1450,  2.1092,  2.0016,   2.129,  2.0346,  2.1651,  2.1145,  2.0786,  2.11295;
  Eigen::VectorXd z_pred(n_z);
  z_pred <<
    6.12155,
     0.245993,
     2.10313;
  Eigen::MatrixXd S(n_z, n_z);
  S <<
      0.0946171, -0.000139448,   0.00407016,
   -0.000139448,  0.000617548, -0.000770652,
     0.00407016, -0.000770652,    0.0180917;
  /*** Perform the UKF Innovation step ***/
  ukf.UpdateState(&x, &P);
  // Perform the L2 norm to compare the output values
  Eigen::VectorXd x_expected(5, 1);
  x_expected <<
    5.92276,
    1.41823,
    2.15593,
    0.489274,
    0.321338;
  Eigen::MatrixXd P_expected(5, 5);
  P_expected <<
    0.00361579, -0.000357881, 0.00208316, -0.000937196, -0.00071727,
   -0.000357881, 0.00539867,  0.00156846,  0.00455342,   0.00358885,
    0.00208316,  0.00156846,  0.00410651,  0.00160333,   0.00171811,
   -0.000937196, 0.00455342,  0.00160333,  0.00652634,   0.00669436,
   -0.00071719,  0.00358884,  0.00171811,  0.00669426,   0.00881797;
  // Precision (i.e., max allowed magnitude of the outputs' L2 difference)
  // NOTE: test works only when all variables above are set manually 
  double epsilon = 0.0001;
  std::cout << "Result `x` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << x.isApprox(x_expected, epsilon) << "\n";
  std::cout << "Result `P` matches expected amount by `epsilon = " << epsilon << '`';
  std::cout << ": " << std::boolalpha << P.isApprox(P_expected, epsilon) << "\n";
}

In [33]:
// Exercise 2.5.6: Innovation Step with State and Covariance Update
test_update_state();

Updated state x: 
 5.92276
 1.41823
 2.15593
0.489274
0.321338
Updated state covariance P: 
  0.00361579 -0.000357881   0.00208316 -0.000937196  -0.00071727
-0.000357881   0.00539867   0.00156846   0.00455342   0.00358885
  0.00208316   0.00156846   0.00410651   0.00160333   0.00171811
-0.000937196   0.00455342   0.00160333   0.00652634   0.00669436
 -0.00071719   0.00358884   0.00171811   0.00669426   0.00881797
Result `x` matches expected amount by `epsilon = 0.0001`: true
Result `P` matches expected amount by `epsilon = 0.0001`: true


## 3. Closing Remarks

##### Alternatives
* Consider different values of the design parameter $\lambda$.

##### Extensions of task
* Apply the UKF algorithm to vehicle tracking with real-world data;
* Use the [Normalised Innovation Squared](https://kalman-filter.com/normalized-innovation-squared/) (NIS) method to measure UKF consistency w.r.t. the design parameters' experimental values.

## 4. Future Work

- ✅ Consider different values of the noise processes $\sigma_{a}^{2}$ and $\sigma_{\ddot{\psi}}^{2}$ for use in e.g., highway / urban / congested traffic environments;
- ⬜️ Consider different values of the sigma point design parameter $\lambda$ for use in e.g., highway / urban / congested traffic environments;
- ⬜️ Use the [Normalised Innovation Squared](https://kalman-filter.com/normalized-innovation-squared/) (NIS) method to measure UKF consistency w.r.t. the design parameters' experimental values.

## Credits

This assignment was prepared by Dominik Nuss, Andrei Vatavu et al., 2020 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).


References
* [1] Kalman, R.E. A New Approach to Linear Filtering and Prediction Problems. Journal of Basic Engineering. 82(1):25-45. 1960. [doi:10.1115/1.3662552](https://doi.org/10.1115/1.3662552).
* [2] Julier, S.J., et al. New Extension of the Kalman Filter to Nonlinear Systems. Proceedings Signal Processing, Sensor Fusion, and Target Recognition. SPIE. 3068(6):182-193. 1997. [doi:10.1117/12.280797](https://doi.org/10.1117/12.280797).
* [3] Wan, E.A., et al. The Unscented Kalman Filter for Nonlinear Estimation. Proceeds of the IEEE 2000 Adaptive Systems for Signal Processing, Communications, and Control (Cat. No.00EX373). 2000. [doi:10.1109/ASSPCC.2000.882463](https://doi.org/10.1109/ASSPCC.2000.882463).
* [4] Li, H. A Brief Tutorial on Recursive Estimation With Examples From Intelligent Vehicle Applications (Part III): Handling Nonlinear Estimation Problems and the Unscented Kalman Filter. 2014. [hal:01054709](https://hal.archives-ouvertes.fr/hal-01054709).

Helpful resources:
* [CTRV Model | Blog post](https://fjp.at/process%20models/model/bicycle-model/ctrv-model/);
* [The Unscented Kalman Filter: Anything EKF can do I can do it better by H. S. Chadha | Medium](https://towardsdatascience.com/the-unscented-kalman-filter-anything-ekf-can-do-i-can-do-it-better-ce7c773cf88d);
* [Mathematical Formula Note of Unscented Kalman Filter with CTRV model | Blog post by @fevemania](https://fevemania.github.io/blog/mathematic-formula-note-of-unscented-kalman-filter/);
* [Normalizing Angles in UKF Robot Localization Implementation — Issue #114 | GitHub](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/issues/114);
* [Normalised Innovation Squared (NIS) by S. Dingler | Kalman filter for Professionals](https://kalman-filter.com/normalized-innovation-squared/).